In [ ]:
import pyodbc
import numpy as np
import onnxmltools
import onnxruntime as rt
import pandas as pd
import skl2onnx
import sklearn
import sklearn.datasets

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=iotedgevm-be9601db-be9601db.eastus.cloudapp.azure.com;'
                      'Database=IoTEdgeDB;'
                      'UID=sa;'
                      'PWD=SuperSecretP@ssw0rd!;')

query = 'SELECT top 100 * FROM dbo.SensorTemperature ORDER BY timeCreated DESC'
cursor = conn.cursor()
df = pd.read_sql(query, conn)

target_column = 'ambientHumidity'
x_train = pd.DataFrame(df.drop(['timeCreated', target_column], axis = 1))
y_train = pd.DataFrame(df.iloc[:, df.columns.tolist().index(target_column)])

print("\n Training dataset x\n")
print(x_train.head())

print("\n Training dataset y\n")
print(y_train.head())

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])
    ]
)

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]
)

model.fit(x_train, y_train)

In [21]:
# Score the model
from sklearn.metrics import r2_score, mean_squared_error
y_pred = model.predict(x_train)
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

*** Scikit-learn r2 score: 0.02572781185295059
*** Scikit-learn MSE: 0.6414608086760171


In [27]:
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType

def convert_dataframe_schema(df, drop=None, batch_axis=False):
    inputs = []
    nrows = None if batch_axis else 1
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([nrows, 1])
        elif v == 'float32':
            t = FloatTensorType([nrows, 1])
        elif v == 'float64':
            t = DoubleTensorType([nrows, 1])
        else:
            raise Exception("Bad type")
        inputs.append((k, t))
    return inputs

In [28]:
# Convert the scikit model to onnx format
onnx_model = skl2onnx.convert_sklearn(model, 'Sensor Temperature', convert_dataframe_schema(x_train), final_types=[('variable1',FloatTensorType([1,1]))])
# Save the onnx model locally
onnx_model_path = 'sensortemp.model.onnx'
onnxmltools.utils.save_model(onnx_model, onnx_model_path)

In [29]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_model_path)

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

onnx_r2_score = r2_score(y_train, y_pred)
onnx_mse = mean_squared_error(y_train, y_pred)

print()
print('*** Onnx r2 score: {}'.format(onnx_r2_score))
print('*** Onnx MSE: {}\n'.format(onnx_mse))
print('R2 Scores are equal' if sklearn_r2_score == onnx_r2_score else 'Difference in R2 scores: {}'.format(abs(sklearn_r2_score - onnx_r2_score)))
print('MSE are equal' if sklearn_mse == onnx_mse else 'Difference in MSE scores: {}'.format(abs(sklearn_mse - onnx_mse)))
print()


*** Onnx r2 score: 0.025739706887990432
*** Onnx MSE: 0.6414529769849469

Difference in R2 scores: 1.1895035039843904e-05
Difference in MSE scores: 7.831691070214752e-06



In [31]:
# Create the model table
table_name = 'Models'

query = f'create table {table_name} ( ' \
    f'[id] [int] IDENTITY(1,1) NOT NULL, ' \
    f'[data] [varbinary](max) NULL, ' \
    f'[description] varchar(1000))'
cursor.execute(query)
conn.commit()

# Insert the ONNX model into the models table
query = f"insert into {table_name} ([description], [data]) values ('Onnx Model',?)"

model_bits = onnx_model.SerializeToString()

insert_params  = (pyodbc.Binary(model_bits))
cursor.execute(query, insert_params)
conn.commit()